In [ ]:
!pip install datasets langchain_community fuzzywuzzy sql_metadata -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 38.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 18.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 2.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
import sqlite3
import sqlparse
import re
from datasets import load_dataset, Dataset
from langchain_community.utilities.sql_database import SQLDatabase
from sql_metadata import Parser
from fuzzywuzzy import process

In [ ]:
from google.colab import userdata
WRITE_TOKEN = userdata.get('hf_write')

In [ ]:
import pandas as pd
pd.set_option('display.max_colwidth', None) #definir a opção para mostrar todo o conteúdo das células
pd.set_option('display.max_rows', None) #definir a opção para mostrar todas as linhas

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
path_base = '/content/drive/Shareddrives/LLMs/Datasets/'

train_e_dev_spider_path = path_base + 'spider/database/' #+ {db_id} + '/' + {db_id} + '.sqlite'
test_spider_path = path_base + 'spider/test_database/'   #+ {db_id} + '/' + {db_id} + '.sqlite'

train_bird_path = path_base + 'bird/train/train_databases/'      #+ {db_id} + '/' + {db_id} + '.sqlite'

In [ ]:
database_path = train_e_dev_spider_path

In [ ]:
def format_dict(dicionario):
    sep = "', '"
    return '{\n  '+",\n  ".join([f"'{chave}': ['{sep.join(valor)}']" for chave, valor in dicionario.items()])+'\n}'

In [ ]:
from fuzzywuzzy import process
def find_closest_match(reference, all):
    closest_match = process.extractOne(reference, all)
    if closest_match:
        return closest_match[0]
    else:
        return None

In [ ]:
def get_schema_dict(db_id):

    db_path = f'{database_path}{db_id}/{db_id}.sqlite'

    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()

    schema_str = "{\n"

    # obter uma lista de todas as tabelas
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
    tables = cursor.fetchall()

    for table in tables:
        table_name=table[0]

        # obter informações das colunas da tabela
        cursor.execute(f"PRAGMA table_info('{table_name}')")
        columns = cursor.fetchall()

        schema_str += f"  '{table_name}': ["

        for column in columns:
            column_name = column[1].replace('"','')
            schema_str += f"'{column_name}', "

        schema_str = schema_str.rstrip(", ") # remover a última vírgula

        schema_str += "],\n"

    schema_str = schema_str.rstrip(",\n")
    schema_str += '\n}'

    conn.close()
    return schema_str

In [ ]:
def get_tables_and_columns(db_id, query, schema_dict, pk=False):

    db_path = f'{database_path}{db_id}/{db_id}.sqlite'
    connection_string="sqlite:///"+db_path
    db = SQLDatabase.from_uri(connection_string, sample_rows_in_table_info=0)

    all_tables = [table.lower() for table in db.get_table_names()]
    tables_in_query = [table.lower() for table in Parser(query).tables]

    #conferir se todas as tabelas fazem parte do schema:
    correct_tables = []
    for table in tables_in_query:
      if table not in all_tables:
        closest_match = find_closest_match(table, all_tables)
        if closest_match:
            if closest_match in all_tables:
              correct_tables.append(closest_match)
      else:
        correct_tables.append(table)

    query = query.replace('(',' ').replace(')',' ').replace(',',' ').replace(';',' ').replace('t1.','').replace('t2.',' ').replace('t3.',' ').replace('t4.',' ')
    query = query.lower().split()
    # print(query)
    tokens = []
    for token in query:
      if '.' in token:
        token = token.split('.')[1] #pegar nome da coluna
      tokens.append(token)
    # print(tokens)
    query = tokens

    info = {}

    for table_name in correct_tables:
      if table_name in query:
        info[table_name] = []

        columns = eval(db.run((f'PRAGMA table_info("{table_name}")')))

        for column in columns:
          column_name = column[1].lower()

          if pk and column[5] == 1:  # se for a incluir chave primária, e a coluna for uma chave primária
            info[table_name].append(column_name) #incluir, mesmo se ela não estiver na query

          else:
            if column_name in query: # or table_name+'.'+column_name in query:
               info[table_name].append(column_name)

    ### correct columns names:
    info_correct = {}

    dict_info = eval(format_dict(info))
    tables_min = [table for table in dict_info.keys()]
    schema_correct = eval(schema_dict)
    all_tables = [table for table in schema_correct.keys()]

    for table in tables_min:
      correct_table = find_closest_match(table,all_tables)
      if correct_table:
        correct_table = correct_table
      else:
        correct_table = table

      correct_columns = []

      for column in dict_info[table]:
        correct_column = find_closest_match(column,schema_correct[correct_table])
        if correct_column:
          correct_columns.append(correct_column)
        else:
          correct_columns.append(column)

      info_correct[correct_table] = correct_columns

    return info, info_correct


In [ ]:
def get_selector(example):
  db_id = example['db_id']
  schema_dict = get_schema_dict(db_id)
  info, info_correct = get_tables_and_columns(db_id, example['query'], schema_dict, True)

  example['schema_dict'] = schema_dict
  example['selector'] = format_dict(info)
  example['selector_correct'] = format_dict(info_correct)

  return example
